<a href="https://colab.research.google.com/github/sunjin7725/SJ-SOLAR-10.7b-DPO/blob/main/merge_lora.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers peft bitsandbytes accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 26.5 MB/s eta 0:00:00


In [ ]:
import torch

from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

base_model_name = 'SJ-Donald/SOLAR-10.7b-slerp'
adapter_model_name = 'SJ-Donald/SJ-SOLAR-10.7b-DPO'

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(base_model_name)
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    return_dict=True,
    torch_dtype=torch.float16,
)

tokenizer_config.json:   0%|          | 0.00/2.81M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.77M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/685 [00:00<?, ?B/s]

...

model-00009-of-00012.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

model-00010-of-00012.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

model-00011-of-00012.safetensors:   0%|          | 0.00/1.94G [00:00<?, ?B/s]

model-00012-of-00012.safetensors:   0%|          | 0.00/201M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/12 [00:00<?, ?it/s]

In [ ]:
model = PeftModel.from_pretrained(base_model, adapter_model_name, device_map='auto')
model = model.merge_and_unload()

adapter_config.json:   0%|          | 0.00/692 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/916M [00:00<?, ?B/s]

In [ ]:
model.save_pretrained('output', safe_serialization=True, max_shard_size="5GB")
tokenizer.save_pretrained('output')

('output/tokenizer_config.json',
 'output/special_tokens_map.json',
 'output/tokenizer.json')

In [ ]:
!pip install -qU huggingface_hub

from huggingface_hub import ModelCard, ModelCardData
from jinja2 import Template

username = "SJ-Donald"
new_model = 'SJ-SOLAR-10.7b-DPO'

template_text = """
---
license: cc-by-nc-4.0
tags:
- DPO
---

# SJ-Donald/SJ-SOLAR-10.7b-DPO

SJ-Donald/SJ-SOLAR-10.7b-DPO is fine-tuned using DPO method.

## Environment

Using **Google CoLab A100**

## Base model

* [SJ-Donald/SOLAR-10.7B-slerp](https://huggingface.co/SJ-Donald/SOLAR-10.7B-slerp)

## Datasets

* [SJ-Donald/orca-dpo-pairs-ko](https://huggingface.co/datasets/SJ-Donald/orca-dpo-pairs-ko)

## How to use

```Python
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

repo = 'SJ-Donald/SJ-SOLAR-10.7b-DPO'

tokenizer = AutoTokenizer.from_pretrained(repo)
model = AutoModelForCausalLM.from_pretrained(
    repo,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map='auto'
)
```
"""

# Create a Jinja template object
jinja_template = Template(template_text.strip())

# Fill the template
content = jinja_template.render()

# Save the model card
card = ModelCard(content)
card.save('output/README.md')

In [ ]:
from google.colab import userdata
from huggingface_hub import HfApi

username = "SJ-Donald"
token = userdata.get('HF_WRITE_TOKEN')
upload_model_name = f"{username}/{new_model}"

# Defined in the secrets tab in Google Colab
api = HfApi(token=token)

# api.create_repo(
#     repo_id=f"{username}/{MODEL_NAME}",
#     repo_type="model"
# )
api.upload_folder(
    repo_id=upload_model_name,
    folder_path='output',
)

model-00002-of-00005.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00003-of-00005.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00004-of-00005.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00001-of-00005.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

Upload 5 LFS files:   0%|          | 0/5 [00:00<?, ?it/s]

model-00005-of-00005.safetensors:   0%|          | 0.00/1.94G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/SJ-Donald/SJ-SOLAR-10.7b-DPO/commit/618e5aedf02d58358d6fda7d9fa67c169b7156d0', commit_message='Upload folder using huggingface_hub', commit_description='', oid='618e5aedf02d58358d6fda7d9fa67c169b7156d0', pr_url=None, pr_revision=None, pr_num=None)